# 蒙特卡洛（MC）强化学习方法笔记
## 一、蒙特卡洛估计基础
蒙特卡洛（Monte Carlo）方法的核心思想是通过大量采样估计随机变量的期望，基于大数定律：当采样数量足够多时，样本平均值会收敛到真实期望。在强化学习中，用于估计状态价值（
v
）或动作价值（
q
），无需依赖环境模型（model-free）。
## 二、三种蒙特卡洛强化学习方法
### 1. MC Basic（基础蒙特卡洛方法）
原理
MC Basic 是最基础的无模型蒙特卡洛方法，核心是将策略迭代（Policy Iteration）中依赖模型的部分替换为无模型采样，直接估计动作价值（
q
值）。
核心步骤
策略评估（Policy Evaluation）
对每个状态 - 动作对
(s,a)
，收集足够多从
(s,a)
出发、遵循当前策略
π 
k
​
 
的 episode，计算这些 episode 的回报（return）平均值，作为
q 
π 
k
​
 
​
 (s,a)
的估计（
q 
k
​
 (s,a)
）。
公式：
q 
k
​
 (s,a)≈ 
N
1
​
 ∑ 
i=1
N
​
 g 
(i)
 (s,a)
，其中
g 
(i)
 (s,a)
是第
i
个 episode 的回报。
策略改进（Policy Improvement）
基于估计的
q 
k
​
 (s,a)
，采用贪心策略更新：对每个状态
s
，选择使
q 
k
​
 (s,a)
最大的动作
a 
∗
 
，新策略
π 
k+1
​
 (a 
∗
 ∣s)=1
（确定性策略）。
优缺点
优点：直观展示无模型蒙特卡洛方法的核心思想，与策略迭代直接对应。
缺点：效率极低（需为每个
(s,a)
单独收集 episode），实际中不实用。
### 2. MC Exploring Starts（带探索起点的蒙特卡洛方法）
原理
MC Exploring Starts 是 MC Basic 的优化，通过高效利用数据和逐 episode 更新提升效率，依赖 “探索起点（Exploring Starts）” 假设。
核心改进
数据高效利用
不再仅使用 episode 的初始
(s,a)
，而是利用 episode 中所有访问过的
(s,a)
对：
First-visit：仅用每个
(s,a)
在 episode 中首次出现后的回报更新
q
值。
Every-visit：用每个
(s,a)
在 episode 中每次出现后的回报更新
q
值。
逐 episode 更新策略
无需等待所有 episode 收集完成，每生成一个 episode 就更新
q
值和策略，符合广义策略迭代（GPI） 框架（策略评估和改进交替进行，不要求精确估计）。
探索起点假设
要求每个
(s,a)
对都能作为 episode 的起点被访问，确保所有
(s,a)
都有足够数据。
核心步骤
随机选择初始状态 - 动作对
(s 
0
​
 ,a 
0
​
 )
（满足探索起点假设），遵循当前策略生成 episode。
从 episode 末尾倒推计算每个
(s 
t
​
 ,a 
t
​
 )
的回报
g
，用 first-visit 或 every-visit 方法更新
q
值。
基于更新后的
q
值，用贪心策略改进策略。
优缺点
优点：数据利用率高，收敛速度快于 MC Basic。
缺点：依赖探索起点假设，实际场景中难以实现（如机器人无法随意设置初始状态 - 动作）。
### 3. MC ε-Greedy（ε- 贪心蒙特卡洛方法）
原理
MC ε-Greedy 去除了探索起点假设，通过软策略（ε-greedy） 平衡探索与利用，确保所有状态 - 动作对能被访问。
核心改进
ε-greedy 策略（软策略）
对每个状态
s
，以概率
1−ε+ε/∣A(s)∣
选择当前最优动作（利用），以概率
ε/∣A(s)∣
选择其他动作（探索），其中
∣A(s)∣
是状态
s
的动作数。
公式：
$$
π(a∣s)={ 
1−ε+ε/∣A(s)∣
ε/∣A(s)∣
​
  
若 a是最优动作
其他动作
​
 
$
$$$
无需探索起点
由于 ε-greedy 策略确保每个动作有正概率被选择，足够长的 episode 可覆盖所有
(s,a)
对，无需强制从每个
(s,a)
出发。
核心步骤
随机选择初始状态 - 动作对，遵循当前 ε-greedy 策略生成足够长的 episode。
用 every-visit 方法（充分利用数据）从 episode 倒推计算回报，更新
q
值。
基于更新后的
q
值，用 ε-greedy 策略改进策略（而非纯贪心）。
优缺点
优点：无需探索起点假设，实际场景中易实现；通过 ε 平衡探索与利用。
缺点：策略最优性受 ε 影响（ε 越大，探索越强但最优性越差），通常需随学习进程减小 ε。
## 三、三种方法对比
方法	核心改进	依赖假设	数据利用率	实用性
MC Basic	无模型估计动作价值	无（但效率极低）	低	理论参考
MC Exploring Starts	高效利用数据、逐 episode 更新	探索起点假设	中	部分场景适用
MC ε-Greedy	ε-greedy 策略平衡探索利用	无（依赖长 episode 覆盖）	高	广泛实用
## 四、关键结论
三种方法均基于蒙特卡洛估计，核心是通过采样回报的平均值估计动作价值。
演进逻辑：从低效基础方法（MC Basic）到高效但有假设（MC Exploring Starts），再到实用无假设（MC ε-Greedy）。
实际应用中，MC ε-Greedy 是主流，通过调整 ε 可平衡探索与利用，适应复杂环境。